In [90]:
# import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials
# from spotipy.oauth2 import SpotifyOAuth
import os
import requests
from dotenv import load_dotenv
import base64
import json
import webbrowser
import urllib
import tqdm
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from dataclasses import dataclass, asdict, fields
from datetime import datetime, date

In [91]:
load_dotenv()
REDIRECT_URI = "http://localhost:8501/"
CLIENT_ID = os.getenv("client_id")
CLIENT_SECRET = os.getenv("client_secret")
SCOPE = 'user-library-read'
SCOPE_USER_TOP_READ = 'user-top-read'

## Request User Authorization

In [92]:
# Create the authorization URL
params = {
    'response_type': 'code',
    'client_id': CLIENT_ID,
    'scope': SCOPE,
    'redirect_uri': REDIRECT_URI,
}
url = f"https://accounts.spotify.com/authorize?{urllib.parse.urlencode(params)}"

# Open the authorization URL in the default web browser
webbrowser.open(url)

True

In [44]:
CODE = "AQDo62KVpKFM_PQ1-lBgL-H118XxN3TY2zFfnMn8aiSskMZFSP9A6I8abu6L8iias_uu71Fj6DMVLkfkcSWIFcItcQowaGN9wq__proPscYUQFsIcj-afXU1dDZHcvbqdnn-H94hdCwvjXEtXPkgkzucMi8FsMxL1tarTsSHK1hJZgV495NN"

## Get Token Info
- Lasts only 1 hour

In [45]:
def get_token_info():
    auth_header =base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode('utf-8')).decode('utf-8')
    url ="https://accounts.spotify.com/api/token"
    headers={
            "Authorization": f"Basic {auth_header}",
            "Content-Type": "application/x-www-form-urlencoded",}
    data={
            "grant_type": "authorization_code",
            "code" : CODE,
            "redirect_uri": REDIRECT_URI}
    result=requests.post(url, headers=headers, data=data)
    json_results=json.loads(result.content)
    return json_results

token_details=get_token_info()

In [46]:
token_details

{'access_token': 'BQAJ5Qh8lvwhlC5n5ClonSx6cNDXCasqaxO2eSbzhtvI3uxyVnMI4e6u6VHMKERGlZSDveEKeNNf1DF_cpZE3lK-0DSGjcjsbF-2dCJ_wvOkOTiLhTMZpQCLhJLbpw8GUeinNruQqSeYyyvjK6YjJBjUn5aaqNptfgRN6QSWsukwrc_ZiLWz9_45JUImFAo',
 'token_type': 'Bearer',
 'expires_in': 3600,
 'refresh_token': 'AQDHwH4bizQ_3UDzfmmjXAiw2cr9v8YhhHfdKQJjbM4weItXbmemhxmhGc9C3QuoxkXXTqo4FjInC-ljtN1QtoziVHMRIW1ADVEGeVOn6e30KZGCXNf6sIm_wnyHILrqKtU',
 'scope': 'user-library-read'}

In [47]:
token = token_details["access_token"]
refresh_token = token_details["refresh_token"]
scope = token_details["scope"]

## Get Auth Header

In [48]:
def get_auth_header(token):
    return f"Bearer " + token

## Post Request
- Request using token for API endpoints

In [9]:
SPOTIFY_GET_CURRENT_USER_SAVED_TRACKS_URL = "https://api.spotify.com/v1/me/tracks?limit=50&offset=0"

In [58]:
def get_current_user_tracks(url):
    response = requests.get(url=url, headers={"Authorization" : f"{get_auth_header(token)}"})
    resp_json = response.json()
    return resp_json

In [59]:
def loop_through():
    first_spotify_url = "https://api.spotify.com/v1/me/tracks?limit=50&offset=0"
    results = get_current_user_tracks(first_spotify_url)
    full_list = []
    current_url = results["href"]
    while results["next"] is not None:
        results = get_current_user_tracks(current_url)
        full_list.append(results)
        spotify_url_next = results["next"]
        current_url = spotify_url_next
    return full_list
            

In [60]:
saved_tracks_data = loop_through()

## Results

In [61]:
saved_tracks_parsed={}
for index_i, i in enumerate(saved_tracks_data):  
    for index_j, j in enumerate(saved_tracks_data[index_i]["items"]):
        saved_tracks_parsed[j["track"]["uri"]] = {
              "added_to_playlist_time":j["added_at"]
            , "artist_name":j["track"]["album"]["artists"][0]["name"]
            , "artist_image":j['track']['album']['images'][1]['url']
            , "unique_artist_id": j["track"]["album"]["artists"][0]["uri"]
            , "release_date": j["track"]["album"]["release_date"]
            , "song_name": j["track"]["name"]
            , "song_length_seconds": int(j["track"]["duration_ms"])/1000
            , "song_explicit": str(j["track"]["explicit"])
            , "song_linkId": j["track"]["external_urls"]["spotify"]
            , "song_popularity": int(j["track"]["popularity"])
            , "album_name": j["track"]["album"]["name"]
            , "spotify_uri_album": j["track"]["album"]["uri"]
            , "spotify_track_id": j["track"]["id"]  
                                            }

In [62]:
saved_tracks_parsed

{'spotify:track:2IGMVunIBsBLtEQyoI1Mu7': {'added_to_playlist_time': '2023-09-22T19:11:14Z',
  'artist_name': 'Doja Cat',
  'artist_image': 'https://i.scdn.co/image/ab67616d00001e027acee948ecac8380c1b6ce30',
  'unique_artist_id': 'spotify:artist:5cj0lLjcoR7YOSnhnX0Po5',
  'release_date': '2023-08-04',
  'song_name': 'Paint The Town Red',
  'song_length_seconds': 231.75,
  'song_explicit': 'True',
  'song_linkId': 'https://open.spotify.com/track/2IGMVunIBsBLtEQyoI1Mu7',
  'song_popularity': 100,
  'album_name': 'Paint The Town Red',
  'spotify_uri_album': 'spotify:album:54dZypaXHAIDzBe9ujAZ63',
  'spotify_track_id': '2IGMVunIBsBLtEQyoI1Mu7'},
 'spotify:track:4Sope1x1EUQiWPyCMKFRRN': {'added_to_playlist_time': '2023-09-02T01:35:57Z',
  'artist_name': 'Grace Grundy',
  'artist_image': 'https://i.scdn.co/image/ab67616d00001e02763ea6b940ff07700fd34243',
  'unique_artist_id': 'spotify:artist:3pFPBG84k5bQVbL9apf2PD',
  'release_date': '2016-11-25',
  'song_name': '(Everything I Do) I Do It for

## Get song genres from Get Track API Endpoint
### Seems as if most songs do not have a genre object/attribute associated with them (shelving this for now)

In [15]:
# for i in full_list_parsed[0:10]:
#     print(i[0])

# full_list_parsed[0]["0.0"][-1].split(":")[-1]
# list(full_list_parsed[0].keys())

In [16]:
# def get_track_genres(url):
#     response = requests.get(url=url, headers={"Authorization" : f"{get_auth_header(token)}"})
#     resp_json = response.json()
#     return resp_json


# get_track_genres("https://api.spotify.com/v1/tracks?ids=6POZ7ocsOQQvJrfArz6KOD")
# for track_id in full_list_parsed:
#     get_track_genre_url= f"https://api.spotify.com/v1/tracks/{track_id}"
#     get_track_genres(get_track_genre_url)

{'tracks': [{'album': {'album_type': 'single',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/757FXqX0Osk2pqtgv4E5v4'},
      'href': 'https://api.spotify.com/v1/artists/757FXqX0Osk2pqtgv4E5v4',
      'id': '757FXqX0Osk2pqtgv4E5v4',
      'name': 'NURKO',
      'type': 'artist',
      'uri': 'spotify:artist:757FXqX0Osk2pqtgv4E5v4'},
     {'external_urls': {'spotify': 'https://open.spotify.com/artist/0p3tzEAt0XWrBqbrwBoN1I'},
      'href': 'https://api.spotify.com/v1/artists/0p3tzEAt0XWrBqbrwBoN1I',
      'id': '0p3tzEAt0XWrBqbrwBoN1I',
      'name': 'Kyle Hume',
      'type': 'artist',
      'uri': 'spotify:artist:0p3tzEAt0XWrBqbrwBoN1I'}],
    'available_markets': ['AR',
     'AU',
     'AT',
     'BE',
     'BO',
     'BR',
     'BG',
     'CA',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DK',
     'DO',
     'DE',
     'EC',
     'EE',
     'SV',
     'FI',
     'FR',
     'GR',
     'GT',
     'HN',
     'HK',
     'HU',
     'IS',
 

## Call Track Auto Features API endpoint
- https://developer.spotify.com/documentation/web-api/reference/get-several-audio-features

In [17]:
def get_tracks_audio_analysis(url):
    response = requests.get(url=url, headers={"Authorization" : f"{get_auth_header(token)}"})
    resp_json = response.json()
    return resp_json

### Creating string for audio track inputs
- Spotify limits the inputs to 100 track ids at a time

In [18]:
listofids=[]
for index, i in enumerate(list(saved_tracks_parsed.values())):
    listofids.append(i["spotify_track_id"])

In [19]:
len(listofids)/100

45.06

## Breaking up Id Lists
- Track Audio Analysis only takes in 100 ids at a time

In [20]:
def list_chunks(l,n):
    for i in range(0,len(l), n):
        yield l[i:i + n]

n = 100

chunksoftrackids=list(list_chunks(listofids,n))

In [21]:
len(chunksoftrackids)

46

In [22]:
string_urls=[]
for index,chunk in enumerate(chunksoftrackids):
    string_urls.append(','.join(chunk))

## Getting Track Audio Features

In [23]:
track_audio_features={}
for index,i in enumerate(string_urls):
    audiofeatures= get_tracks_audio_analysis(f'https://api.spotify.com/v1/audio-features?ids={string_urls[index]}')
    for j in audiofeatures["audio_features"]:
        track_audio_features[j['uri']] ={
                "danceability":j["danceability"]
                , "energy":j["energy"]
                , "key":j["key"]
                , "loudness":j["loudness"]
                , "mode":j["mode"]
                , "speechiness":j["speechiness"]
                , "acousticness":j["acousticness"]
                , "instrumentalness":j["instrumentalness"]
                , "liveness":j["liveness"]
                , "valence":j["valence"]
                , "time_signature":j["time_signature"]
                , "tempo":j["tempo"]
                                        }

In [24]:
track_details_all = {k: {**saved_tracks_parsed[k], **track_audio_features[k]} for k in saved_tracks_parsed}

In [25]:
len(track_details_all)

4506

In [26]:
# @dataclass
# class Song:
#     song_id: str
#     added_to_playlist_time: datetime
#     artist_name: str
#     unique_artist_id: str
#     release_date: date
#     song_name: str
#     song_length_seconds: int
#     song_explicit: bool
#     song_linkId: str
#     song_popularity: int
#     album_name: str
#     spotify_uri_album: str
#     spotify_track_id: str
#     danceability: float
#     energy: float
#     key: int
#     loudness: float
#     mode: float
#     speechiness: float
#     acousticness: float
#     instrumentalness: float
#     liveness: float
#     valence: float
#     time_signature: int
#     tempo: float

In [27]:
# dataclass_song_data =[Song(k, **v) for k,v in track_details_all.items()]

In [28]:
temp=list(track_details_all.values())

In [29]:
track_details_df=pd.DataFrame(temp)

In [40]:
track_details_df.head()

,added_to_playlist_time,artist_name,artist_image,unique_artist_id,release_date,song_name,song_length_seconds,song_explicit,song_linkId,song_popularity,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,time_signature,tempo
0,2023-09-22T19:11:14Z,Doja Cat,https://i.scdn.co/image/ab67616d00001e027acee9...,spotify:artist:5cj0lLjcoR7YOSnhnX0Po5,2023-08-04,Paint The Town Red,231.750,True,https://open.spotify.com/track/2IGMVunIBsBLtEQ...,100,...,5,-8.603,1,0.1740,0.26900,0.000003,0.0901,0.732,4,99.968
1,2023-09-02T01:35:57Z,Grace Grundy,https://i.scdn.co/image/ab67616d00001e02763ea6...,spotify:artist:3pFPBG84k5bQVbL9apf2PD,2016-11-25,(Everything I Do) I Do It for You,123.283,False,https://open.spotify.com/track/4Sope1x1EUQiWPy...,49,...,0,-8.686,1,0.0368,0.89600,0.000000,0.0961,0.243,4,128.841
2,2023-08-23T20:03:00Z,Jeremy Zucker,https://i.scdn.co/image/ab67616d00001e024f491a...,spotify:artist:3gIRvgZssIb9aiirIg0nI3,2023-06-09,i need you (in my life),189.106,True,https://open.spotify.com/track/5NMl4DvH391bWzv...,49,...,8,-7.650,1,0.0307,0.10100,0.000062,0.1160,0.625,4,130.015
3,2023-08-21T23:04:49Z,Parannoul,https://i.scdn.co/image/ab67616d00001e028ef3be...,spotify:artist:7eZbNxarrTW4VkRI8u9aDX,2023-01-28,Polaris,262.500,False,https://open.spotify.com/track/6Yt8PGw2VEbLH5p...,36,...,6,-5.812,1,0.0351,0.00259,0.001750,0.1200,0.158,4,80.015
4,2023-08-21T23:04:07Z,Caroline Polachek,https://i.scdn.co/image/ab67616d00001e02de1789...,spotify:artist:4Ge8xMJNwt6EEXOzVXju9a,2023-02-14,Billions,297.514,False,https://open.spotify.com/track/4JPF8Te7Jo3AuI2...,50,...,5,-7.892,1,0.1090,0.16900,0.000067,0.0994,0.361,4,139.939


In [30]:
track_details_df.to_csv('song_data.csv', sep=',', index=False)

In [31]:
# track_details_df.head()

In [32]:
# corr = track_details_df.select_dtypes(include=[np.int64, float]).corr()
# Create a heatmap
# plt.figure(figsize=(10,8)) 
# sns.heatmap(corr.round(2), annot=True, cmap='coolwarm')

In [33]:
# print(track_details_df.columns)
# track_details_df.groupby(["artist_name"])["artist_name"].count().sort_values(ascending=False)[0:11]
# This only counts them if they are the main artist

In [34]:
# (track_details_df["song_explicit"].value_counts()/len(track_details_df))*100

In [35]:
# track_details_df["added_to_playlist_month"] = pd.to_datetime(track_details_df.added_to_playlist_time).dt.strftime('%Y-%m')

In [36]:
# plt.figure(figsize=(25, 10))
# plt.bar(data=track_details_df.groupby("added_to_playlist_month").count().sort_values(by=["added_to_playlist_month"])["song_name"], x=track_details_df.groupby("added_to_playlist_month").count().sort_values(by=["added_to_playlist_month"])["song_name"].index, height=track_details_df.groupby("added_to_playlist_month").count().sort_values(by=["added_to_playlist_month"])["song_name"].values)

In [37]:
# temp = pd.DataFrame(pd.read_csv('song_data.csv'))
# temp.head()

,added_to_playlist_time,artist_name,artist_image,unique_artist_id,release_date,song_name,song_length_seconds,song_explicit,song_linkId,song_popularity,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,time_signature,tempo
0,2023-09-22T19:11:14Z,Doja Cat,https://i.scdn.co/image/ab67616d00001e027acee9...,spotify:artist:5cj0lLjcoR7YOSnhnX0Po5,2023-08-04,Paint The Town Red,231.750,True,https://open.spotify.com/track/2IGMVunIBsBLtEQ...,100,...,5,-8.603,1,0.1740,0.26900,0.000003,0.0901,0.732,4,99.968
1,2023-09-02T01:35:57Z,Grace Grundy,https://i.scdn.co/image/ab67616d00001e02763ea6...,spotify:artist:3pFPBG84k5bQVbL9apf2PD,2016-11-25,(Everything I Do) I Do It for You,123.283,False,https://open.spotify.com/track/4Sope1x1EUQiWPy...,49,...,0,-8.686,1,0.0368,0.89600,0.000000,0.0961,0.243,4,128.841
2,2023-08-23T20:03:00Z,Jeremy Zucker,https://i.scdn.co/image/ab67616d00001e024f491a...,spotify:artist:3gIRvgZssIb9aiirIg0nI3,2023-06-09,i need you (in my life),189.106,True,https://open.spotify.com/track/5NMl4DvH391bWzv...,49,...,8,-7.650,1,0.0307,0.10100,0.000062,0.1160,0.625,4,130.015
3,2023-08-21T23:04:49Z,Parannoul,https://i.scdn.co/image/ab67616d00001e028ef3be...,spotify:artist:7eZbNxarrTW4VkRI8u9aDX,2023-01-28,Polaris,262.500,False,https://open.spotify.com/track/6Yt8PGw2VEbLH5p...,36,...,6,-5.812,1,0.0351,0.00259,0.001750,0.1200,0.158,4,80.015
4,2023-08-21T23:04:07Z,Caroline Polachek,https://i.scdn.co/image/ab67616d00001e02de1789...,spotify:artist:4Ge8xMJNwt6EEXOzVXju9a,2023-02-14,Billions,297.514,False,https://open.spotify.com/track/4JPF8Te7Jo3AuI2...,50,...,5,-7.892,1,0.1090,0.16900,0.000067,0.0994,0.361,4,139.939


## Getting Most associated Artists and Tracks

### Request user Auth

In [71]:
# Create the authorization URL
params = {
    'response_type': 'code',
    'client_id': CLIENT_ID,
    'scope': SCOPE_USER_TOP_READ,
    'redirect_uri': REDIRECT_URI,
}
url = f"https://accounts.spotify.com/authorize?{urllib.parse.urlencode(params)}"

# Open the authorization URL in the default web browser
webbrowser.open(url)

True

## USER TOP TOKEN

In [72]:
CODE="AQCsnUhHCJ6AJtdXUEugPvT7Mbw4LmBpXIA2TLoZB82G5hnPp85Yf9eduOcT_pAqkqKbrEMfD6m4GuvjnG8qWoNvPr-jn6zuZXdqAEIsI5bWxsLiQb4QbfCACq1k3naVGS_zX9dFCmbf-YHE7ASIQNkp0BlFBk1vVCzkTmVblfogBXs"

### USER TOP SCOPE TOKEN

In [73]:
def get_token_info():
    auth_header =base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode('utf-8')).decode('utf-8')
    url ="https://accounts.spotify.com/api/token"
    headers={
            "Authorization": f"Basic {auth_header}",
            "Content-Type": "application/x-www-form-urlencoded",}
    data={
            "grant_type": "authorization_code",
            "code" : CODE,
            "redirect_uri": REDIRECT_URI}
    result=requests.post(url, headers=headers, data=data)
    json_results=json.loads(result.content)
    return json_results

token_details=get_token_info()

In [80]:
USER_TOP_ITEM_SCOPE_TOKEN=token_details["access_token"]

In [81]:
def get_top_user_items(url_items):
    response = requests.get(url=url_items, headers={"Authorization" : f"{get_auth_header(USER_TOP_ITEM_SCOPE_TOKEN)}"})
    resp_json = response.json()
    return resp_json

In [82]:
url_items = "https://api.spotify.com/v1/me/top/artists?limit=50&offset=0&time_range=long_term"
results_top_user_items = get_top_user_items(url_items)

In [86]:
results

{'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4MCBfE4596Uoi2O4DtmEMz'},
   'followers': {'href': None, 'total': 0},
   'genres': ['chicago rap', 'melodic rap', 'rap'],
   'href': 'https://api.spotify.com/v1/artists/4MCBfE4596Uoi2O4DtmEMz',
   'id': '4MCBfE4596Uoi2O4DtmEMz',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab6761610000e5eb1908e1a8b79abf71d5598944',
     'width': 640},
    {'height': 320,
     'url': 'https://i.scdn.co/image/ab676161000051741908e1a8b79abf71d5598944',
     'width': 320},
    {'height': 160,
     'url': 'https://i.scdn.co/image/ab6761610000f1781908e1a8b79abf71d5598944',
     'width': 160}],
   'name': 'Juice WRLD',
   'popularity': 86,
   'type': 'artist',
   'uri': 'spotify:artist:4MCBfE4596Uoi2O4DtmEMz'},
  {'external_urls': {'spotify': 'https://open.spotify.com/artist/6Xgp2XMz1fhVYe7i6yNAax'},
   'followers': {'href': None, 'total': 0},
   'genres': ['melodic rap', 'rap', 'trap'],
   'href': 'https://api.spo

In [89]:
results.get("items")[0]

{'external_urls': {'spotify': 'https://open.spotify.com/artist/4MCBfE4596Uoi2O4DtmEMz'},
 'followers': {'href': None, 'total': 0},
 'genres': ['chicago rap', 'melodic rap', 'rap'],
 'href': 'https://api.spotify.com/v1/artists/4MCBfE4596Uoi2O4DtmEMz',
 'id': '4MCBfE4596Uoi2O4DtmEMz',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb1908e1a8b79abf71d5598944',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab676161000051741908e1a8b79abf71d5598944',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f1781908e1a8b79abf71d5598944',
   'width': 160}],
 'name': 'Juice WRLD',
 'popularity': 86,
 'type': 'artist',
 'uri': 'spotify:artist:4MCBfE4596Uoi2O4DtmEMz'}